## Capstone Project

In [1]:
# install packages
# coding: utf-8
!pip install unidecode

In [2]:
# download corpus of stop words
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yumengli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# import modules
import joblib
import numpy as np
import pandas as pd
import plotly.express as px
import scipy.stats as ss
import unidecode
from nltk.corpus import stopwords
from scipy.special import inv_boxcox
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [4]:
# load the data
data = pd.read_csv("pet food data.csv")

There are 684 entries that do not have price data, 548 entries that do not have an ingredients list, 755 entries that do not have a total package size, and 747 entries that do not have a unit package size.

In [5]:
# get a description of the unprocessed data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8754 entries, 0 to 8753
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  8754 non-null   object 
 1   category              8754 non-null   object 
 2   sub_category          8754 non-null   object 
 3   product               8754 non-null   object 
 4   variant               8754 non-null   object 
 5   description           8754 non-null   object 
 6   ingredients_stdlist   8206 non-null   object 
 7   price_usd             8070 non-null   float64
 8   company_parent        8754 non-null   object 
 9   company               8754 non-null   object 
 10  brand                 8754 non-null   object 
 11  total_pack_size_ml_g  7999 non-null   float64
 12  unit_pack_size_ml_g   8007 non-null   float64
dtypes: float64(3), object(10)
memory usage: 889.2+ KB


In [6]:
# determine if there are any duplicate rows
np.sum(data.duplicated())

0

In [7]:
data.company.unique

<bound method Series.unique of 0                         Mars
1         Big Heart Pet Brands
2                         Iams
3                         Iams
4       Del Monte Pet Products
                 ...          
8749                    Target
8750     Nestlé Purina PetCare
8751      Big Heart Pet Brands
8752     Nestlé Purina PetCare
8753      Big Heart Pet Brands
Name: company, Length: 8754, dtype: object>

### Initial Cleaning for Visualization

In [8]:
# drop all of the NaN entries
data.dropna(inplace=True)

# remove 1 outlier price with invalid pack size
data = data[data.price_usd < 50]

# remove 1 unit pack size equal to 0
data = data[data.unit_pack_size_ml_g > 0]

# drop 32 therapeutic supplements 
data = data.loc[~data.ingredients_stdlist.str.startswith("active"), :]

# reset the index
data.reset_index(drop=True, inplace=True)

In [9]:
# transform the string dates into datetime objects and create a year column
data.loc[:, "date"] = pd.to_datetime(data.date.str[:10])
data["year"] = data.date.apply(lambda x: x.year)

In [10]:
# drop the "category" column since it contains one value "Pet Food" and
# the "variant" column since it is equivalent to the "product" column
data.drop(columns=["category", "product"], inplace=True)

In [11]:
# add a ratio column of pack sizes to the dataframe
data["ratio"] = data.total_pack_size_ml_g / data.unit_pack_size_ml_g

In [12]:
# transform the "sub_category" column into separate columns and drop "sub_category"
data["pet_type"] = data.sub_category.apply(lambda x: "cat" if ("Cat" in x) else "dog")
data["meal_type"] = data.sub_category.apply(lambda x: "primary" if ("Food" in x) else "treats")
data["food_type"] = data.sub_category.apply(lambda x: "dry" if ("Dry" in x) else "wet")
data.drop(columns="sub_category", inplace=True)

In [13]:
# create a function to replace dicritic characters with ASCII characters
def replace_dicritic(string):
    return unidecode.unidecode(string)

In [14]:
# replace the dicritic characters in "company_parent", "company", "brand",
# "description", and "ingredients_stdlist"
data.loc[:, "company_parent"] = data.company_parent.apply(replace_dicritic)
data.loc[:, "company"] = data.company.apply(replace_dicritic)
data.loc[:, "brand"] = data.brand.apply(replace_dicritic)
data.loc[:, "description"] = data.description.apply(replace_dicritic)
data.loc[:, "ingredients_stdlist"] = data.ingredients_stdlist.apply(replace_dicritic)

In [15]:
# get a description of the cleaned data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7525 entries, 0 to 7524
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  7525 non-null   datetime64[ns]
 1   variant               7525 non-null   object        
 2   description           7525 non-null   object        
 3   ingredients_stdlist   7525 non-null   object        
 4   price_usd             7525 non-null   float64       
 5   company_parent        7525 non-null   object        
 6   company               7525 non-null   object        
 7   brand                 7525 non-null   object        
 8   total_pack_size_ml_g  7525 non-null   float64       
 9   unit_pack_size_ml_g   7525 non-null   float64       
 10  year                  7525 non-null   int64         
 11  ratio                 7525 non-null   float64       
 12  pet_type              7525 non-null   object        
 13  meal_type         

### Visualizations

In [16]:
# get the observations for Colgate only
colgate_only = data[data.company_parent.apply(lambda x: "colgate" in x.lower())]

In [17]:
# perform a Box-Cox transformation on price_usd
box_cox = ss.boxcox(data.price_usd)

Boxplots

In [18]:
# boxplot for cat vs dog
fig1 = px.box(data, x="pet_type", y="price_usd")
fig2 = px.box(colgate_only, x="pet_type", y="price_usd", category_orders={"pet_type": ["cat", "dog"]})

# boxplot for primary food vs treats
fig3 = px.box(data, x="meal_type", y="price_usd")
fig4 = px.box(colgate_only, x="meal_type", y="price_usd")

# boxplot for dry vs wet food
fig5 = px.box(data, x="food_type", y="price_usd")
fig6 = px.box(colgate_only, x="food_type", y="price_usd", category_orders={"food_type": ["dry", "wet"]})

# boxplot for year
fig7 = px.box(data, x="year", y="price_usd")

Histograms

In [19]:
# histogram of price
fig8 = px.histogram(data.price_usd)

# histogram of price with Box-Cox transformation
fig9 = px.histogram(box_cox[0], nbins=50)

Scatter Plots

In [20]:
# scatter plot of price vs unit pack size
fig10 = px.scatter(data.loc[~data.unit_pack_size_ml_g.isna(), :], x="unit_pack_size_ml_g", y="price_usd", trendline="ols", trendline_color_override="orange", range_y=[-2, 37])

### Text Transformations

In [21]:
# get the first word in every ingredient list
first_ingredient = data.ingredients_stdlist.str.split(",").apply(lambda x: x[0])
first_ingredient = first_ingredient.str.split(" ").apply(lambda x: x[0])
first_ingredient = first_ingredient.str.split(":").apply(lambda x: x[0].lower())

In [22]:
# look at the top 10 words by frequency count
first_ingredient.value_counts()[:10]

chicken    1871
wheat       675
beef        568
corn        412
water       350
turkey      289
salmon      249
lamb        225
rice        203
duck        115
Name: ingredients_stdlist, dtype: int64

In [23]:
# create a function that filters the first_ingredient Series to
# 5 possible words
def first_word(string):
    if string == "chicken" or string == "wheat" or string == "beef" or string == "corn":
        return string
    else:
        return "other"

In [24]:
# add a first_ingredient column to the dataset
data["first_ingredient"] =  first_ingredient.apply(first_word)

TFIDF

During testing, we discovered that using a TFIDF matrix with more than ~400 features in our regression models caused instability, resulting in errors that were $10^6$ or greater. To improve performance, we set max_features=400 in our TfidfVectorizer().

In [25]:
# instantiate the TFIDF vectorizer
vectorizer = TfidfVectorizer(stop_words=stopwords.words("english"), max_features=400)

# transform the product descriptions in a TFIDF matrix
vectors = vectorizer.fit_transform(data.description)

# convert the TFIDF matrix into a dataframe
tfidf = pd.DataFrame(vectors.todense().tolist(), columns=vectorizer.get_feature_names())

# manually remove any remaining stop words
tfidf.drop(columns=["10", "100", "12", "13", "14", "15", "16", "20", "24", "25", "50", "also", "lb", "lbs", "oz"], inplace=True)

Convert categorical variables to one-hot vectors

In [26]:
# get all of the categorical variables in one dataframe
cat_vars = data[["pet_type", "meal_type", "food_type", "first_ingredient"]]

# transform the categorical variables into one-hot vectors
one_hot = pd.get_dummies(cat_vars, prefix="category")

### Combine all of the engineered features and create training and testing sets

In [27]:
# gather all of the pack size columns
sizes = data[["total_pack_size_ml_g", "unit_pack_size_ml_g", "ratio"]]
sizes

,total_pack_size_ml_g,unit_pack_size_ml_g,ratio
0,3175.20,3175.20,1.0
1,10886.40,10886.40,1.0
2,4445.28,4445.28,1.0
3,4898.88,4898.88,1.0
4,6123.60,6123.60,1.0
...,...,...,...
7520,708.75,708.75,1.0
7521,708.75,708.75,1.0
7522,708.75,708.75,1.0
7523,708.75,708.75,1.0


In [28]:
# combine the 1hot dataframe with the TFIDF dataframe
X = pd.concat([one_hot, sizes, tfidf], axis=1)

# create a target variable for the Box-Cox transformed prices
y = box_cox[0]

In [29]:
# create stratified training and testing sets
# box-cox prices
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=data.pet_type, random_state=38)

# original prices
X_train_org, X_test_org, y_train_org, y_test_org = train_test_split(X, data.price_usd, train_size=0.8, stratify=data.pet_type, random_state=38)

In [30]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6020, 399), (1505, 399), (6020,), (1505,))

In [31]:
X_train_org.shape, X_test_org.shape, y_train_org.shape, y_test_org.shape

((6020, 399), (1505, 399), (6020,), (1505,))

In [32]:
X.shape, y.shape

((7525, 399), (7525,))

### Modeling

Train a Random Forest Regressor using a cross-validated random grid search.

In [33]:
# set up a parameter grid for random grid search
# max number of levels per tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)

# min number of samples required at each leaf node
min_samples_leaf = [int(x) for x in np.linspace(5, 50, num = 10)]

# number of trees in the random forest
n_estimators = [int(x) for x in np.linspace(100, 1000, num = 10)]

random_grid = {"max_depth": max_depth,
               "max_features": ["auto", "sqrt"],
               "min_samples_leaf": min_samples_leaf,
               "n_estimators": n_estimators}
random_grid

{'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
 'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}

In [ ]:
# instantiate a RandomForestRegressor
forest_reg = RandomForestRegressor()

# instantiate a RandomizedSearchCV
random_search = RandomizedSearchCV(forest_reg, random_grid,
                                   n_iter=100, cv=5,
                                   scoring="neg_mean_squared_error")

# conduct a random search
random_search.fit(X_train_org, y_train_org)

# view the results of our random search
random_search.best_params_

In [ ]:
# get the predicted y values
ypred = random_search.best_estimator_.predict(X_test_org)

# calculate the RMSE of the RandomForestRegressor model
rmse = mean_squared_error(y_test_org, ypred, squared=False)
rmse

Train a Random Forest Regressor using a cross-validated grid search.

In [ ]:
# set up a parameter grid for grid search
search_grid = {"max_depth": [40, 45, 50, 55, 60],
               "min_samples_leaf": [3],
               "n_estimators": [700, 750, 800, 850, 900]}
search_grid

In [ ]:
# instantiate a RandomForestRegressor
forest_reg = RandomForestRegressor()

# instantiate a GridSearchCV
grid_search = GridSearchCV(forest_reg, search_grid,
                             cv=5, n_jobs=6,
                             scoring="neg_mean_squared_error")

# conduct a grid search
grid_search.fit(X_train_org, y_train_org)

# view the results of our grid search
grid_search.best_params_

In [ ]:
# get the predicted y values
ypred = grid_search.best_estimator_.predict(X_test_org)

# calculate the RMSE of the RandomForestRegressor model
rmse = mean_squared_error(y_test_org, ypred, squared=False)
rmse

Polynomial regression with Box-Cox transformed prices

In [ ]:
# define a custom polynomial regression function
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

In [ ]:
# set up a parameter grid for grid search
search_grid = {"polynomialfeatures__degree": np.arange(1, 21),
               "linearregression__fit_intercept": [True, False],
               "linearregression__normalize": [True, False]}
search_grid

In [ ]:
# instantiate a GridSearchCV
grid_search = GridSearchCV(PolynomialRegression(), search_grid,
                           cv=5, n_jobs=6,
                           scoring="neg_mean_squared_error")

# conduct a grid search
grid_search.fit(X_train, y_train)

# view the results of our grid search
grid_search.best_params_

In [ ]:
# get the predicted y values
ypred = grid_search.best_estimator_.predict(X_test)

# calculate the RMSE of the RandomForestRegressor model
rmse = mean_squared_error(y_test, ypred, squared=False)

# reverse the box-cox transformation to get the rmse in dollars
inv_boxcox(rmse, box_cox[1])

Standardize the training and testing data

In [ ]:
# standardize the training and testing data
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.fit_transform(X_test)

Ridge regression with Box-Cox transformed prices

In [ ]:
# set up a parameter grid for grid search
search_grid = {"alpha": np.logspace(-6, 4, 11),
               "fit_intercept": [True, False]}

# instantiate a Ridge
ridge_reg = Ridge()

# instantiate a GridSearch
grid_search = GridSearchCV(ridge_reg, search_grid,
                           cv=5, n_jobs=6,
                           scoring="neg_mean_squared_error")

# conduct a grid search
grid_search.fit(X_train_std, y_train)

# view the results of our grid search
grid_search.best_params_

In [ ]:
# get the predicted y values
ypred = grid_search.best_estimator_.predict(X_test_std)

# calculate the RMSE of the RandomForestRegressor model
rmse = mean_squared_error(y_test, ypred, squared=False)

# reverse the box-cox transformation to get the rmse in dollars
inv_boxcox(rmse, box_cox[1])

LASSO regression with Box-Cox transformed prices

In [ ]:
# set up a parameter grid for grid search
search_grid = {"alpha": np.logspace(-3, -1, 100)}
               #"fit_intercept": [True, False]}

# instantiate a Lasso
lasso_reg = Lasso()

# instantiate a GridSearch
grid_search = GridSearchCV(lasso_reg, search_grid,
                           cv=5, n_jobs=6,
                           scoring="neg_mean_squared_error")

# conduct a grid search
grid_search.fit(X_train_std, y_train)

# view the results of our grid search
grid_search.best_params_

In [ ]:
# get the predicted y values
ypred = grid_search.best_estimator_.predict(X_test_std)

# calculate the RMSE of the RandomForestRegressor model
rmse = mean_squared_error(y_test, ypred, squared=False)

# reverse the box-cox transformation to get the rmse in dollars
inv_boxcox(rmse, box_cox[1])

ElasticNet regression with Box-Cox transformed prices

In [ ]:
# set up a parameter grid for grid search
search_grid = {"alpha": np.logspace(-6, 4, 11),
               "l1_ratio": np.linspace(0.2, 0.8, 4)}

# instantiate a ElasticNet
elastic_reg = ElasticNet()

# instantiate a GridSearch
grid_search = GridSearchCV(elastic_reg, search_grid,
                           cv=5, n_jobs=6,
                           scoring="neg_mean_squared_error")

# conduct a grid search
grid_search.fit(X_train_std, y_train)

# view the results of our grid search
grid_search.best_params_

In [ ]:
# get the predicted y values
ypred = grid_search.best_estimator_.predict(X_test_std)

# calculate the RMSE of the RandomForestRegressor model
rmse = mean_squared_error(y_test, ypred, squared=False)

# reverse the box-cox transformation to get the rmse in dollars
inv_boxcox(rmse, box_cox[1])

### Predictive Price Modeling

Optimal Model Implementation

In [ ]:
# standardize the dataset
scaler = StandardScaler()
X_std = scaler.fit_transform(X)
X_std.shape

In [ ]:
# instantiate a LASSO regressor with optimal hyperparameters
lasso_reg = Lasso(alpha=0.006428)

# train the LASSO regressor on the full dataset
lasso_reg.fit(X_std, y)

In [ ]:
# get the predicted prices
pred = lasso_reg.predict(X_std)

# reverse the box-cox transformation to get the predicted prices in dollars
pred_usd = inv_boxcox(pred, box_cox[1])

Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
# put the coefficients in a Series with their respective names
coef = pd.Series(lasso_reg.coef_, index=X.columns.values)

# sort the values in place, descending
coef.sort_values(ascending=False, inplace=True)

# get the top 20 and bottom 20
top20_bottom20 = pd.concat([coef[:20], coef[-20:]])

In [ ]:
plt.figure(figsize=(9, 12))
sns.barplot(top20_bottom20.values, top20_bottom20.index.values,
            orient="h", palette='coolwarm')
plt.xlim(-0.5, 0.7)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title("Top 20 Largest and Bottom 20 Smallest Coefficients", fontsize=14);

In [ ]:
plt.figure(figsize=(16, 9))
plt.scatter(np.arange(0, 7525), pred_usd, alpha=0.5, label="Predicted")
plt.scatter(np.arange(0, 7525), data.price_usd, alpha=0.25, label="Actual")
plt.legend();

## Grouping  Models

### PCA Analysis

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X_std)
PCA(n_components=2)
principalComponents = pca.fit_transform(X_std)

In [ ]:
principalDF = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'] )
principalDF

PCA for meal type

In [ ]:
finalDF_mealType = pd.concat([principalDF, data[["meal_type"]]], axis = 1)
finalDF_mealType

In [ ]:
# visualizing PCA
fig = plt.figure(figsize = (8, 8))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ["treats", "primary"]
colors = ['r', 'g']
for target, color in zip(targets, colors):
    indx = finalDF_mealType['meal_type'] == target
    ax.scatter(finalDF_mealType.loc[indx, 'principal component 1'],
              finalDF_mealType.loc[indx, 'principal component 2'],
             c = color, s = 50)
ax.legend(targets)
ax.grid()

PCA for foodtype

In [ ]:
finalDF_foodType = pd.concat([principalDF, data[["food_type"]]], axis = 1)
finalDF_foodType

In [ ]:
fig = plt.figure(figsize = (8, 8))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ["dry", "wet"]
colors = ['r', 'g']
for target, color in zip(targets, colors):
    indx = finalDF_foodType['food_type'] == target
    ax.scatter(finalDF_foodType.loc[indx, 'principal component 1'],
              finalDF_foodType.loc[indx, 'principal component 2'],
             c = color, s = 50)
ax.legend(targets)
ax.grid()

### Doc2Vec Analysis Using Cosine Similarity 
#### this shows how similar a new description document is to the other existing documents

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

In [ ]:
import gensim
import smart_open
## Using Gensim package reading in the corpus
def read_corpus(set):       
    for i, line in enumerate(set):
        tokens = gensim.utils.simple_preprocess(line)
        yield gensim.models.doc2vec.TaggedDocument(tokens, [i])
corpus = list(read_corpus(data.description))

In [ ]:
# Building the doc to vec model 
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(corpus)

In [ ]:
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
# Pick a random document from the corpus and infer a vector from the model
String ='Tibbles & Bits brand organic dog food is naturally gluten and grain-free, and provides your pet with the best variety of raw ingredients. Follow us on Facebook to see all of our cookies, chews, and jerky.'
tokens = gensim.utils.simple_preprocess(String)
inferred_vector = model.infer_vector(tokens)
## choose the top 100 most similar documents with biggest to smallest cosine smilarity score
sims = model.docvecs.most_similar([inferred_vector], topn=100)
sims

In [ ]:
# get the index of the most similar documents from the tuple sims object
# first is index second is cosine similarity score
inds=[i[0] for i in sims]
# create a new dataframe with these new index of top 100 most similar documents
data_new = data.loc[inds]
data_new.head()

In [ ]:
# get the top 5 most similar documents with the given index 
top5 = data_new[:5]
# get the top 5 most similar pet food descriptions
print(top5.description)
# get the top 5 most similar pet food company
print(top5.company)

In [ ]:
# get the summary statistic of the sales price of top 10 most similar documents
data_new[:10].price_usd.describe()